### Tokenizer

In [1]:
tokenized_text = "Jim Henson was a puppeteer".split()
tokenized_text

['Jim', 'Henson', 'was', 'a', 'puppeteer']

Its too simple. So we need to use subword tokenization.

In [6]:
from transformers import BertTokenizer

tokenizer_bert= BertTokenizer.from_pretrained("bert-base-cased")

In [13]:
from transformers import AutoTokenizer
tokenizer_auto = AutoTokenizer.from_pretrained("bert-base-cased")

In [8]:
tokenizer_bert("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenizer_auto("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
sequence = "Using a Transformer network is simple"
tokens = tokenizer_auto.tokenize(sequence)
tokens

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']

In [19]:
ids = tokenizer_auto.convert_tokens_to_ids(tokens)
ids

[7993, 170, 13809, 23763, 2443, 1110, 3014]

In [20]:
decoded_string = tokenizer_auto.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


### Handling a sequence of inputs

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [30]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
# This line will fail.
try:
    model(input_ids)
except Exception as e:
    print(e)


In [26]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


Batching

In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids, ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [32]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

# this cannot be converted to a tensor so we use padding

padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

input_ids = [
    [200, 200, 200],
    [200, 200, tokenizer_auto.pad_token_id]
]

model(torch.tensor(input_ids)).logits

tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)

In [36]:
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

In [39]:
output = model(torch.tensor(input_ids), attention_mask=torch.tensor(attention_mask))
output.logits

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)

In [41]:
sequence1 = ["I’ve been waiting for a HuggingFace course my whole life.", "I hate this so much!"]

input_ids = []
for seq in sequence1:
    tokens = tokenizer.tokenize(sequence)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    print(ids)
    input_ids.append(ids)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [43]:
model(torch.tensor(input_ids)).logits

tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)

In [44]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

In [45]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)


In [46]:
output.logits

tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>)